<a href="https://colab.research.google.com/github/team-signpin/mono/blob/master/Online_Signature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
import numpy as np
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt

In [ ]:
import os

# [x[0] for x in os.walk("./")]
#for x in next(os.walk('./drive/MyDrive/Work/Thesis/MOBISIG/USER1'))[2]:
# print(x)

In [ ]:
import pandas as pd
signature1 = pd.read_csv('/content/drive/MyDrive/Work/Thesis/MOBISIG/USER1/SIGN_FOR_USER1_USER2_2.csv')
signature1.head()

In [ ]:
s1_coords = signature1.iloc[:,:2]
s1_coords

In [ ]:
s1_coords.plot()

## Fetch all signatures related to a single user



In [ ]:
# list of all users files
from os import listdir
user_list = listdir("./drive/MyDrive/Work/Thesis/MOBISIG")
user_list = user_list[2:] # removes stats and users file
user_list = [int(x[4:]) for x in user_list]
user_list.sort()
user_list = ["USER" + str(x) for x in user_list]
user_list[:5]

## Fetch all signatures related to a single user

In [ ]:
# fetch all file names of user_id
def all_files_related_to(user_id):
  user_list = listdir("./drive/MyDrive/Work/Thesis/MOBISIG/USER" + str(user_id) + "/")
  return user_list

all_files_related_to(1)[:5]

## Fetched all the original signatures

In [ ]:
# array of original signatures
user_id = 1
orig_sigs = []
for i in range(1, 46):
  sig_data = pd.read_csv(f"/content/drive/MyDrive/Work/Thesis/MOBISIG/USER1/SIGN_GEN_USER{user_id}_USER{user_id}_" + str(i) +'.csv')
  sig_data = sig_data.iloc[:, :3]
  orig_sigs.append(sig_data)

# pd.read_csv('/content/drive/MyDrive/Work/Thesis/MOBISIG/USER1/SIGN_GEN_USER1_USER1_1.csv')
print("Shape of a signature", orig_sigs[0].shape)

## Normalized the signature into fixed number of data points

In [ ]:
import math
def distance(p1, p2):
  return math.sqrt( ((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2) )


def midDistance(data, n):
  dis = data[:, 3]
  given_di = n
  end_point = np.searchsorted(dis, given_di)
  end_point_vdi = data[end_point, 3]
  start_point_vdi = data[end_point - 1, 3]
  start_d_point = data[end_point-1];
  end_d_point = data[end_point]
  div = (given_di - start_point_vdi) / (end_point_vdi - start_point_vdi)

  new_point_x = (end_d_point[0] - start_d_point[0])*div + start_d_point[0]
  new_point_y = (end_d_point[1] - start_d_point[1])*div + start_d_point[1]
  
  new_point = [new_point_x, new_point_y, start_d_point[2], start_d_point[3], given_di]
  new_point = [round(x, 5) for x in new_point]
  return new_point

def cumulativeDistance(data):
  # FIXME more accurate, print to see the issue in the beginning and ending
  data = np.pad(data, ((0, 0), (0, 1)), mode='constant', constant_values=0)
  c_data = data
  sum = 0
  for i in range(0, len(data) -1):
    c_data[i][3] = sum
    next_point = [c_data[i+1][0], c_data[i+1][1]]
    current_point = [c_data[i][0], c_data[i][1]]
    sum += distance(current_point, next_point)
  return c_data


def normalizeByDistance(data, n):
  c_data = cumulativeDistance(data)
  # print("c_data", c_data.shape)
  total_distance = c_data[-2][3]
  # print(f'total distance: {total_distance}')
  return np.array([ midDistance(c_data, (total_distance/n) * x) for x in range(1,n)])


def normalizeSign(orig_sig):
  tmp_np = np.array(orig_sig)
  tmp_normalized = normalizeByDistance(tmp_np, 91)
  return tmp_normalized / 1200


tmp = normalizeSign(orig_sigs[0])
plt.plot(tmp[:,0], tmp[:,1], "bo")
plt.plot(tmp[:,0], tmp[:,1])
print("shape of a signature", tmp.shape)

## Fetch forged signatures

In [ ]:
# List all signature files of a user
from os import listdir
user_sign_path = "./drive/MyDrive/Work/Thesis/MOBISIG/USER" + str(user_id)
sign_list = listdir(user_sign_path)
sig_data_dict = {}
for sign in sign_list:
  tmp_sig_data = pd.read_csv(f"{user_sign_path}/{sign}")
  sig_data_dict[sign] = tmp_sig_data.iloc[:, :3]

len(sig_data_dict)

In [ ]:
# Normalize and label
normalized_signs_dict = {}
for key, value in sig_data_dict.items():
  # print(key[5:8], value.shape)
  try:
    normalized_signs_dict[key] = normalizeSign(value)
  except IndexError as e:
    print(key, e)


len(normalized_signs_dict)

In [ ]:
# trying to plot a normalized sign


# tmp = normalized_signs[3]
last = []
for key, tmp in normalized_signs_dict.items():
  # plt.plot(tmp[:,0], tmp[:,1], "bo")
  tmp = tmp[:,:3]
  last = tmp
  try:
    plt.plot(tmp[:,0], tmp[:,1])
  except TypeError as e:
    print(e)

len(last)

Reshaped

In [ ]:
# Reshaped
reshaped_signs_dict = {}

for key, value in normalized_signs_dict.items():
  tmp = value[:, :3]
  reshaped_signs_dict[key] = tmp.reshape(90*3)

reshaped_signs_dict.keys()
reshaped_signs_dict['SIGN_GEN_USER1_USER1_26.csv'][0]

## Preparing labels

In [1]:
# Preparing labels

tmp_labels = reshaped_signs_dict.keys()
tmp_labels = list(tmp_labels)
labels = []

for label in tmp_labels:
  print(label[5:8])
  if label[5:8] == "FOR":
    labels.append(0)
  elif label[5:8]

SyntaxError: ignored

### Create a table like the following


| signature dataframe | label | means |
| --- | --- | :-: |
| SIG[] | 0 | FORGED |
| SIG[] | 1 | ORIGINAL |
| SIG[] | 0 | FORGED |
| SIG[] | 0 | FORGED |

Here signature dataframe is a matrix of a single signature attempt.

## Converting all signatures to normalise

In [ ]:
plt.plot(orig_sigs[0].iloc[:, 0], orig_sigs[0].iloc[:, 1], 'ro')

In [ ]:
for orig_sig in orig_sigs:
  plt.plot(orig_sig.iloc[:,0], orig_sig.iloc[:,1])
  print(len(orig_sig))

# plt.plot(sig_to_vis.iloc[:,0], sig_to_vis.iloc[:,1])
# plt.plot(sig_to_vis1.iloc[:,0], sig_to_vis1.iloc[:,1])

In [ ]:
import tensorflow as tf